# Imports

In [3]:
# Add package root to the path
import sys
from pathlib import Path
path_root = Path().resolve().parents[0]
sys.path.append(str(path_root))
path_root

PosixPath('/Users/daniel/Idan/University/Masters/Thesis/2024/radar_sort')

In [18]:
import argparse
import json
import torch
from torch.utils.data import DataLoader

from utils.paths_collector import Paths
from data.carrada.dataset import Carrada
from evaluation.tester import Tester
from mvrss.utils.functions import count_params
# from mvrss.learners.tester import Tester
from mvrss.models import TMVANet, MVNet
# from mvrss.loaders.dataset import Carrada
from mvrss.loaders.dataloaders import SequenceCarradaDataset
import json
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

from mvrss.utils.functions import transform_masks_viz, get_metrics, normalize, define_loss, get_transformations, get_qualitatives
# from mvrss.utils.paths import Paths
from mvrss.utils.metrics import Evaluator
from mvrss.loaders.dataloaders import CarradaDataset

from utils.paths_collector import Paths

In [19]:
# PARAMETERS
args = {
    'cfg': '/Users/daniel/Idan/University/Masters/Thesis/2024/datasets/logs/carrada/mvnet/mvnet_e300_lr0.0001_s42_0/config.json',
}

seq_name = '2019-09-16-12-55-51'  # None for all

frame_num_plot = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
    cfg_path = args["cfg"]
    with open(cfg_path, 'r') as fp:
        cfg = json.load(fp)

In [21]:
paths = Paths().get()

exp_name = cfg['name_exp'] + '_' + str(cfg['version'])
path = paths['logs'] / cfg['dataset'] / cfg['model'] / exp_name
model_path = path / 'results' / 'model.pt'
test_results_path = path / 'results' / 'test_results.json'
paths

{'warehouse': PosixPath('/Users/daniel/Idan/University/Masters/Thesis/2024/datasets'),
 'logs': PosixPath('/Users/daniel/Idan/University/Masters/Thesis/2024/datasets/logs'),
 'carrada': PosixPath('/Users/daniel/Idan/University/Masters/Thesis/2024/datasets/Carrada'),
 'config': '/Users/daniel/Idan/University/Masters/Thesis/2024/radar_sort/configs/config.ini'}

In [22]:
iteration=None
get_quali=True
add_temp=True

visualizer = None
cfg = cfg
visualizer = visualizer
model = cfg['model']
nb_classes = cfg['nb_classes']
annot_type = cfg['annot_type']
process_signal = cfg['process_signal']
w_size = cfg['w_size']
h_size = cfg['h_size']
n_frames = cfg['nb_input_channels']
batch_size = cfg['batch_size']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
custom_loss = cfg['custom_loss']
transform_names = cfg['transformations'].split(',')
norm_type = cfg['norm_type']
paths = Paths().get()
test_results = dict()
cfg

{'name': 'mvnet',
 'model': 'mvnet',
 'dataset': 'carrada',
 'signal_type': 'mimo',
 'process_signal': True,
 'annot_format': 'light_frame',
 'annot_type': 'dense',
 'w_size': 256,
 'h_size': 256,
 'nb_classes': 4,
 'nb_input_channels': 3,
 'nb_epochs': 300,
 'batch_size': 15,
 'lr': 0.0001,
 'lr_step': 20,
 'loss_step': 100,
 'val_step': 2000,
 'viz_step': 4000,
 'torch_seed': 42,
 'numpy_seed': 42,
 'version': 0,
 'device': 'cuda',
 'custom_loss': 'wce_w10sdice',
 'transformations': 'flip',
 'norm_type': 'tvt',
 'rd_loss_weight': 1,
 'ra_loss_weight': 1,
 'shuffle': True,
 'comments': 'Multi-View network (MV-Net), former H-Net. Methods: data aug (flip) + multi loss + 3 input frames. Model selection: mean of precision. Normalisation: TVT. Loss: wCE + weighted Soft Dice Loss (10*SDice).',
 'name_exp': 'mvnet_e300_lr0.0001_s42'}

In [25]:

# Construct the path to the frames (assumes paths is defined with key 'carrada')
path_to_frames = paths['carrada'] / seq_name[0]
# Prepare transformations for the test split (assumes get_transformations is defined)
transformations = get_transformations(
    transform_names, split='test', sizes=(w_size, h_size))
seq = [('000050',), ('000051',), ('000052',), ('000053',), ('000054',), ('000055',), ('000056',), ('000059',), ('000060',), ('000061',), ('000062',), ('000063',), ('000064',), ('000065',), ('000069',), ('000070',)]
# Create a DataLoader for the current sequence frames (assumes CarradaDataset is defined)
frame_dataloader = DataLoader(
    CarradaDataset(seq, annot_type, path_to_frames,
                    process_signal, n_frames, transformations, add_temp),
    shuffle=False,
    batch_size=1,
    num_workers=4
)

In [26]:
# Get frame_num_plot from the dataloader
for frame_num, frame in enumerate(frame_dataloader):
    rd_mask = frame['rd_mask'].to(device).float()
    curent_frame = frame_num * len(rd_mask)

    if rd_mask[0][1:].max() > 0.0:
        frame_num_plot -= 1
        print(
            f"frame number: {frame_num}, max value = {rd_mask[0][1:].max()} yeyy")
        if frame_num_plot == 0:
            break

# Retrieve tensors and move them to the device (assumes proper keys exist in frame)
rd_data = frame['rd_matrix'].to(device).float()
ra_data = frame['ra_matrix'].to(device).float()
# Only used if model is 'tmvanet'
ad_data = frame['ad_matrix'].to(device).float()
rd_mask = frame['rd_mask'].to(device).float()
ra_mask = frame['ra_mask'].to(device).float()

# Normalize the inputs (assumes normalize is defined)
rd_data = normalize(rd_data, 'range_doppler', norm_type=norm_type).squeeze()
ra_data = normalize(ra_data, 'range_angle', norm_type=norm_type).squeeze()


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/daniel/miniconda3/envs/radar_sort/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/daniel/miniconda3/envs/radar_sort/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daniel/miniconda3/envs/radar_sort/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/Users/daniel/Idan/University/Masters/Thesis/2024/radar_sort/external/MVRSS/mvrss/loaders/dataloaders.py", line 111, in __getitem__
    rd_m